## Overview

Hello! Welcome to the architecture setup notebook, where we will be installing all requirements and outline the basic architecture of our AlexNet model (whose performance will be compared to our custom model, EfficentNet, and ConvNeXt). 


The cell below handles our initial requirements installation:

In [1]:
!pip3 install -r ../../requirements.txt

  Using cached transformers-4.41.1-py3-none-any.whl (9.1 MB)
  Using cached safetensors-0.4.3-cp38-cp38-macosx_10_12_x86_64.whl (416 kB)
  Using cached regex-2024.5.15-cp38-cp38-macosx_10_9_x86_64.whl (281 kB)
  Using cached tokenizers-0.19.1-cp38-cp38-macosx_10_12_x86_64.whl (2.5 MB)
  Using cached huggingface_hub-0.23.2-py3-none-any.whl (401 kB)
  Using cached fsspec-2024.5.0-py3-none-any.whl (316 kB)


## Data Preprocessing

As part of our data preprocessing, we will split the down-scaled lung dataset from the original dataset into a train/test split. 

Note that we will be using five-fold cross-validation for testing later, hence we will not be partioning an additional validation set. 

After splitting our data, we will then feed the training set into our models. Here, we will specifically feed it into the AlexNet model. 

In [ ]:
# Initialize AlexNet Model (Pending)
model = models.alexnet(pretrained=True)
model = model.to(device)

In [ ]:
# Implement five-fold cross validation (Pending)
num_folds = 5
k_choices = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]

X_train_folds = []
y_train_folds = []
################################################################################
# Split up the training data into folds. After splitting, X_train_folds and    #
# y_train_folds should each be lists of length num_folds, where                #
# y_train_folds[i] is the label vector for the points in X_train_folds[i].     #
# Hint: Look up the numpy array_split function.                                #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

X_train_folds = np.array_split(X_train, num_folds)
Y_train_folds = np.array_split(y_train, num_folds)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# A dictionary holding the accuracies for different values of k that we find
# when running cross-validation. After running cross-validation,
# k_to_accuracies[k] should be a list of length num_folds giving the different
# accuracy values that we found when using that value of k.
k_to_accuracies = {}


################################################################################
# Perform k-fold cross validation to find the best value of k. For each        #
# possible value of k, run the k-nearest-neighbor algorithm num_folds times,   #
# where in each case you use all but one of the folds as training data and the #
# last fold as a validation set. Store the accuracies for all fold and all     #
# values of k in the k_to_accuracies dictionary.                               #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

for k in k_choices:

  k_to_accuracies[k] = []

  for i in range(num_folds):
    # Generate validation set
    validation_set = (X_train_folds[i], Y_train_folds[i])

    # Generate training set
    x_training = np.concatenate([X_train_folds[j] for j in range(num_folds) if j != i])
    y_training = np.concatenate([Y_train_folds[j] for j in range(num_folds) if j != i])

    # Train the classifier
    classifier.train(x_training, y_training)

    # Test classifier and save accuracy
    predictions = classifier.predict(X=validation_set[0], k=k)
    accuracy = np.sum(predictions == validation_set[1]) / len(predictions)
    k_to_accuracies[k].append(accuracy)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# Print out the computed accuracies
for k in sorted(k_to_accuracies):
    for accuracy in k_to_accuracies[k]:
        print('k = %d, accuracy = %f' % (k, accuracy))